# Load and test trained model

## Load libraries

In [1]:
import numpy as np
import torch

import torch.nn as nn
from enduro_lstm import *

In [2]:
device = conf_cuda(False)

Selected CPU


In [3]:
device

device(type='cpu')

In [4]:
device.type

'cpu'

## Set configurations

In [5]:
data_path = r"../1-generate/data/"
match = 35
use_cuda = False
zigzag = False
is_softmax = True

## Load trained model

In [6]:
path = 'models/zigzag_chunk_m35to35_f1to120_epoch1000_H200/'
newpath = path + 'novo_teste_m35to35_f1to120_epoch1000_H200'

In [7]:
zigzag = True

In [8]:
hidden_neurons = 200
load_checkpoint = True

In [9]:
if zigzag:
    output_size = 2
else:
    output_size = 9

In [10]:
def load_checkpoint(model, filename='checkpoint.pth.tar'):
    
    print("=> loading checkpoint '{}'".format(filename))
    checkpoint = torch.load(filename, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])

    return model

In [11]:
if load_checkpoint:
    model = Model(device=device, input_size=12000, output_size=output_size, hidden_dim=hidden_neurons, n_layers=1, is_softmax=is_softmax)
    model = load_checkpoint(model, newpath)
else:
    model = Model(device=device, input_size=12000, output_size=output_size, hidden_dim=hidden_neurons, n_layers=1, is_softmax=is_softmax)
    model.load_state_dict(torch.load(newpath, map_location=device))
model.eval()

=> loading checkpoint 'models/zigzag_chunk_m35to35_f1to120_epoch1000_H200/novo_teste_m35to35_f1to120_epoch1000_H200'


Model(
  (lstm): LSTM(12000, 200, batch_first=True)
  (fc): Linear(in_features=200, out_features=2, bias=True)
  (out): Softmax(dim=None)
)

In [12]:
ACTIONS_LIST = get_actions_list(zigzag=zigzag)

num_of_frames_arr = []
frames_arr = []
actions_arr = []

start_frame = 1
end_frame = 1000

## Prepare cell with trained model 

In [13]:
lstmcell = nn.LSTMCell(12000, hidden_neurons)
linear = nn.Linear(hidden_neurons, output_size)
if is_softmax:
    output = nn.Softmax()
else:
    output = nn.Sigmoid()

In [14]:
output

Softmax(dim=None)

In [15]:
lstmcell.weight_ih = model.lstm.weight_ih_l0
lstmcell.weight_hh = model.lstm.weight_hh_l0
lstmcell.bias_hh = model.lstm.bias_hh_l0
lstmcell.bias_ih = model.lstm.bias_ih_l0
linear.weight = model.fc.weight
linear.bias = model.fc.bias

In [16]:
hx = torch.zeros(1, hidden_neurons)
cx = torch.zeros(1, hidden_neurons)

In [17]:
print(model.lstm.weight_ih_l0.shape)
print(model.lstm.weight_hh_l0.shape)
print(model.lstm.bias_ih_l0.shape)
print(model.lstm.bias_hh_l0.shape)
print(model.fc.weight.shape)
print(model.fc.bias.shape)

torch.Size([800, 12000])
torch.Size([800, 200])
torch.Size([800])
torch.Size([800])
torch.Size([2, 200])
torch.Size([2])


## Play Gym Enduro

In [18]:
import gym
import time
from PIL import Image

In [19]:
if zigzag:
        
    ACTIONS = {
        "right": 2,
        "left": 3,
    }

else:

    ACTIONS = {
        "noop": 0,
        "accelerate": 1,
        "right": 2,
        "left": 3,
        "break": 4,
        "right_break": 5,
        "left_break": 6,
        "right_accelerate": 7,
        "left_accelerate": 8,
    }

In [20]:
y_min, y_max, x_min, x_max = 25+30, 195-40, 20, 140
shape_of_single_frame = (1, (y_max-y_min),(x_max-x_min))

In [21]:
sleep_time = 0.05

In [22]:
env = gym.make("Enduro-v0")
frame = env.reset()
reward, action, done, info = 0, 0, False, {'ale.lives': 0}

hx = torch.zeros(1, hidden_neurons)
cx = torch.zeros(1, hidden_neurons)

env.render()

for _ in range(1000):
    
    time.sleep(sleep_time)
    env.render()
    
    frame = frame[y_min:y_max, x_min:x_max]

    frame = Image.fromarray(frame)
    frame = frame.convert("L")
    
    frame = np.asarray(frame)
    frame = frame.reshape(1, -1)
    frame = torch.tensor(frame)/255
    
    hx, cx = lstmcell(frame, (hx, cx))
    out = linear(hx)
    action = output(out)
    
    action = list(ACTIONS.values())[torch.argmax(action, axis=1)]
    print(action)
    frame, reward, done, info = env.step(action)

<ipython-input-22-d9ca9208c3bd>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  action = output(out)


2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
3
3
3
2
2
2
2
2
2
2
3
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
2
3
3
3
3
3
3
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
3
3
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
3
3
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
2
2
3
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
3
3
3
3
3
2
2
2
2
2
2
3
3
3
3
2
2
2
2
3
3
3
3
3
2
2
2
2
2
2
2
2
2
2
2


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ryo/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-22-d9ca9208c3bd>", line 12, in <module>
    time.sleep(sleep_time)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ryo/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ryo/.local/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/ryo/.local/lib/python3.8/site-packages/IPython/core/

TypeError: object of type 'NoneType' has no len()